In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
import json
import cv2
import numpy as np

class KeyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage,
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data(index)
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] = 224.0 / w
        kps[1::2] = 224.0 / h
        return img, kps

In [ ]:
train_dataset = KeyPointsDataset("data/images",
                                 "data/data_train.json")
val_dataset = KeyPointsDataset("data/images",
                                 "data/data_val.json")

train_dataloader = DataLoader(train_dataset,
                              batch_Size=0, shuffle=True)
test_dataloader = DataLoader(val_dataset, batch_size=0, shuffle=True)

In [ ]:
model = models.resnet(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # replace the last layer (14 kps with (x, y) coord)

model = model.to(device)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [ ]:
epochs = 20
for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)

        output = model(X)
        loss = loss_fn(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            print(f"epoch: {epoch} | loss: {loss.item()}")


In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")